# Getting Started with fastcpd: Part 2 - Detecting Change Points

This tutorial shows how to use fastcpd to detect change points in time series data.

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from fastcpd import fastcpd
from fastcpd.datasets import make_mean_change, make_glm_change, make_garch_change

%matplotlib inline

## Basic Usage

The `fastcpd()` function detects change points. You need to specify:
1. Your data
2. The model `family`
3. Optional: penalty parameter `beta`

## 1. Mean Changes

Simplest case - detecting when the mean shifts.

In [ ]:
# Generate data with STRONG mean changes for clear detection
# Specify: change points at [100, 200, 250] and large mean shifts (5 std devs)
data_dict = make_mean_change(
    n_samples=300, 
    n_changepoints=3,
    mean_deltas=[5.0],  # 5 std deviation shifts - very clear signal
    seed=42
)
data = data_dict['data']
true_cps = data_dict['changepoints']

# Detect change points
result = fastcpd(data, family="mean", beta="MBIC")

print("True change points:", true_cps)
print("Detected change points:", result.cp_set)
print(f"SNR: {data_dict['metadata']['snr_db']:.1f} dB (higher is easier to detect)")

# Visualize
plt.figure(figsize=(12, 4))
plt.plot(data, linewidth=0.8, label='Data')
for cp in true_cps:
    plt.axvline(cp, color='green', linestyle='--', alpha=0.6, label='True' if cp == true_cps[0] else '')
for cp in result.cp_set:
    plt.axvline(cp, color='red', linestyle=':', linewidth=2, alpha=0.8, label='Detected' if cp == result.cp_set[0] else '')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Mean Change Detection')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 2. Variance Changes

Detecting when volatility/variance changes.

In [ ]:
from fastcpd.datasets import make_variance_change

# Generate data with LARGE variance changes for clear detection
# Use variance ratios of [1.0, 4.0, 0.5] for strong contrast
data_dict = make_variance_change(
    n_samples=300, 
    n_changepoints=2,
    variance_ratios=[1.0, 4.0, 0.5],  # Strong variance changes
    seed=42
)
data = data_dict['data']
true_cps = data_dict['changepoints']

# Detect
result = fastcpd(data, family="variance", beta="MBIC")

print("True:", true_cps)
print("Detected:", result.cp_set)
print(f"Variance ratios: {data_dict['metadata']['variance_ratios']}")

## 3. Regression (with Covariates)

When you have predictors/features and want to detect coefficient changes.

In [ ]:
from fastcpd.datasets import make_regression_change

# Generate data
data_dict = make_regression_change(n_samples=300, n_changepoints=2, n_features=3, seed=42)
data = data_dict['data']  # Shape: (n, features+1) - first column is response
true_cps = data_dict['changepoints']

# Detect
result = fastcpd(data, family="lm", beta="BIC")

print("True:", true_cps)
print("Detected:", result.cp_set)

# Visualize response variable
plt.figure(figsize=(12, 4))
plt.plot(data[:, 0], linewidth=0.8)
for cp in true_cps:
    plt.axvline(cp, color='green', linestyle='--', alpha=0.6)
for cp in result.cp_set:
    plt.axvline(cp, color='red', linestyle=':', linewidth=2)
plt.xlabel('Time')
plt.ylabel('Response')
plt.title('Regression: Coefficient Changes')
plt.grid(True, alpha=0.3)
plt.show()

## 4. GLM: Binomial (Binary/Count Data)

For logistic regression or binomial data.

In [ ]:
# Generate binary data with STRONG coefficient changes
# Use larger coefficients for better separation
data_dict = make_glm_change(
    n_samples=300, 
    n_changepoints=2, 
    n_features=3, 
    family='binomial',
    coef_changes='sign_flip',  # Clear coefficient pattern
    seed=42
)
data = data_dict['data']
true_cps = data_dict['changepoints']

# Detect - use vanilla_percentage=1.0 (PELT)
result = fastcpd(data, family="binomial", beta="MBIC", vanilla_percentage=1.0)

print("True:", true_cps)
print("Detected:", result.cp_set)
print(f"AUC per segment: {[f'{a:.2f}' if a else 'N/A' for a in data_dict['metadata']['separation_per_segment']]}")

# Visualize
plt.figure(figsize=(12, 4))
plt.plot(data[:, 0], 'o', markersize=2, alpha=0.6)
for cp in true_cps:
    plt.axvline(cp, color='green', linestyle='--', alpha=0.6)
for cp in result.cp_set:
    plt.axvline(cp, color='red', linestyle=':', linewidth=2)
plt.xlabel('Time')
plt.ylabel('Binary Response')
plt.title('Binomial GLM Detection')
plt.grid(True, alpha=0.3)
plt.show()

## 5. GLM: Poisson (Count Data)

For count/rate data.

In [ ]:
# Generate count data with clear coefficient changes
data_dict = make_glm_change(
    n_samples=300, 
    n_changepoints=2, 
    n_features=3, 
    family='poisson',
    coef_changes='random',  # Random works better than sign_flip for Poisson
    seed=42
)
data = data_dict['data']
true_cps = data_dict['changepoints']

# Detect - use vanilla_percentage=1.0 (PELT) for Poisson
result = fastcpd(data, family="poisson", beta="MBIC", vanilla_percentage=1.0)

print("True:", true_cps)
print("Detected:", result.cp_set)
print(f"Overdispersion per segment: {[f'{o:.1f}' for o in data_dict['metadata']['overdispersion_per_segment']]}")

## 6. ARMA Time Series

For time series with autoregressive and moving average components. We'll use ARMA(1,1) with large coefficient changes for clear detection.

**Note:** ARMA detection can take 20-30 seconds for n=150.

In [ ]:
# Generate ARMA(1,1) data with LARGE coefficient changes
from statsmodels.tsa.arima_process import ArmaProcess

np.random.seed(42)
true_cps = [50, 100]

# Segment 1: ARMA(1,1) with ar=[0.8], ma=[0.6]
ar1 = np.array([1, -0.8])  # AR polynomial
ma1 = np.array([1, 0.6])   # MA polynomial
arma1 = ArmaProcess(ar1, ma1)
y1 = arma1.generate_sample(nsample=50, scale=1.0)

# Segment 2: ARMA(1,1) with ar=[-0.7], ma=[-0.5] (BIG change!)
ar2 = np.array([1, 0.7])
ma2 = np.array([1, -0.5])
arma2 = ArmaProcess(ar2, ma2)
y2 = arma2.generate_sample(nsample=50, scale=1.0)

# Segment 3: ARMA(1,1) with ar=[0.5], ma=[0.3]
ar3 = np.array([1, -0.5])
ma3 = np.array([1, 0.3])
arma3 = ArmaProcess(ar3, ma3)
y3 = arma3.generate_sample(nsample=50, scale=1.0)

data = np.concatenate([y1, y2, y3])

# Detect change points using ARMA(1,1) model
# ARMA models require order parameter [p, q] where p=AR order, q=MA order
result = fastcpd(data, family='arma', order=[1, 1], beta='MBIC')

print(f"True change points: {true_cps}")
print(f"Detected change points: {result.cp_set.tolist()}")
print(f"Coefficients: Seg1=(AR:0.8, MA:0.6), Seg2=(AR:-0.7, MA:-0.5), Seg3=(AR:0.5, MA:0.3)")

In [ ]:
# Generate GARCH(1,1) data with EXTREME volatility changes
from arch import arch_model

np.random.seed(42)
true_cps = [50, 100]

# Segment 1: VERY LOW volatility (omega=0.001, alpha=0.05, beta=0.1)
model1 = arch_model(None, vol='GARCH', p=1, q=1, mean='Zero', rescale=False)
params1 = np.array([0.001, 0.05, 0.1])  # [omega, alpha, beta]
sim1 = model1.simulate(params1, nobs=50)
y1 = sim1['data'].values

# Segment 2: EXTREMELY HIGH volatility (omega=5.0, alpha=0.4, beta=0.4)
model2 = arch_model(None, vol='GARCH', p=1, q=1, mean='Zero', rescale=False)
params2 = np.array([5.0, 0.4, 0.4])
sim2 = model2.simulate(params2, nobs=50)
y2 = sim2['data'].values

# Segment 3: MEDIUM volatility (omega=0.1, alpha=0.2, beta=0.3)
model3 = arch_model(None, vol='GARCH', p=1, q=1, mean='Zero', rescale=False)
params3 = np.array([0.1, 0.2, 0.3])
sim3 = model3.simulate(params3, nobs=50)
y3 = sim3['data'].values

data = np.concatenate([y1, y2, y3])

# Detect change points using GARCH(1,1) model with MBIC
# GARCH models require order parameter [p, q] for GARCH(p,q)
result = fastcpd(data, family='garch', order=[1, 1], beta='MBIC')

print(f"True change points: {true_cps}")
print(f"Detected change points: {result.cp_set.tolist()}")
print(f"Volatility: Seg1=VERY LOW (ω=0.001), Seg2=EXTREME (ω=5.0), Seg3=MEDIUM (ω=0.1)")

## 7. GARCH Volatility Models

For detecting changes in volatility/heteroskedasticity. We'll use GARCH(1,1) with extreme volatility changes.

**Note:** GARCH detection can take 20-30 seconds for n=150.

## 5. Controlling Data Generation

You can control signal strength to make changes easier or harder to detect.

## Controlling Data Generation

You can control signal strength to make changes easier or harder to detect:

**Mean changes:**
```python
# Weak signal (hard to detect)
data_dict = make_mean_change(n_samples=300, n_changepoints=2, mean_deltas=[1.0])

# Strong signal (easy to detect)
data_dict = make_mean_change(n_samples=300, n_changepoints=2, mean_deltas=[5.0])
```

**Variance changes:**
```python
# Subtle changes
data_dict = make_variance_change(n_samples=300, variance_ratios=[1.0, 1.5, 2.0])

# Strong changes  
data_dict = make_variance_change(n_samples=300, variance_ratios=[1.0, 4.0, 0.5])
```

**GLM (Binomial):**
```python
# 'sign_flip' creates clear coefficient pattern (easier to detect)
data_dict = make_glm_change(n_samples=300, family='binomial', coef_changes='sign_flip')
```

**Tip:** Use stronger signals when learning. Once familiar, try weaker signals to test robustness.

In [ ]:
# Generate data with STRONG signal
data_dict = make_mean_change(n_samples=300, n_changepoints=3, mean_deltas=[5.0], seed=42)
data = data_dict['data']

# Try different beta values
for beta in ["MBIC", "BIC", 5.0, 20.0]:
    result = fastcpd(data, family="mean", beta=beta)
    beta_str = str(beta) if isinstance(beta, (int, float)) else beta
    print(f"Beta={beta_str:>6s}: {len(result.cp_set)} change points detected at {result.cp_set}")

### Vanilla Percentage (Speed vs Accuracy)

Controls the trade-off between speed and accuracy:
- `vanilla_percentage=0.0` - Pure SeGD (faster, approximate)
- `vanilla_percentage=1.0` - Pure PELT (slower, exact)
- `vanilla_percentage=0.5` - Hybrid (balanced)

In [ ]:
# For small datasets (n < 500): use vanilla_percentage=1.0 for highest accuracy
result = fastcpd(data, family="mean", beta="MBIC", vanilla_percentage=1.0)

# For larger datasets: use vanilla_percentage=0.5 for speed
# result = fastcpd(data, family="mean", beta="MBIC", vanilla_percentage=0.5)

## Summary

**Model families covered in this tutorial:**
- `"mean"` - Mean changes
- `"variance"` - Variance changes
- `"meanvariance"` - Both mean and variance
- `"lm"` - Linear regression
- `"lasso"` - LASSO regression with L1 regularization
- `"binomial"` - Logistic/binomial GLM
- `"poisson"` - Poisson GLM
- `"arma"` - ARMA time series (requires `order=[p, q]`)
- `"garch"` - GARCH volatility (requires `order=[p, q]`)

**Basic syntax:**
```python
result = fastcpd(data, family="mean", beta="MBIC")
change_points = result.cp_set
```

**Key tips:**
- Use automatic penalty selection (`beta="MBIC"` or `beta="BIC"`) for reliable results
- For GLM models (binomial/poisson), use `vanilla_percentage=1.0` for best accuracy
- ARMA and GARCH require the `order` parameter (e.g., `order=[1, 1]` for ARMA(1,1))
- Use larger coefficient/parameter changes for more reliable detection

---

**Next**: Part 3 - Evaluating and Visualizing Results

In [ ]:
result = fastcpd(data, family="mean", beta="MBIC")

print("Detected change points:", result.cp_set)
print("Number of segments:", len(result.cp_set) + 1)
print("\nAll attributes:")
print(dir(result))

## Summary

**Core model families covered in this tutorial:**
- `"mean"` - Mean changes
- `"variance"` - Variance changes
- `"meanvariance"` - Both mean and variance
- `"lm"` - Linear regression
- `"binomial"` - Logistic/binomial GLM

**Additional families available** (see advanced examples):
- `"lasso"` - LASSO regression
- `"poisson"` - Poisson GLM  
- `"garch"` - GARCH volatility
- `"arma"` - ARMA time series

**Basic syntax:**
```python
result = fastcpd(data, family="mean", beta="MBIC")
change_points = result.cp_set
```

**Best practice**: Use automatic penalty selection (`beta="MBIC"` or `beta="BIC"`) for reliable results.

---

**Next**: Part 3 - Evaluating and Visualizing Results